In [10]:
import os 
import uproot 
from matplotlib import pyplot as plt 
import numpy as np 
import awkward as ak

In [2]:
d = "/eos/user/a/atishelm/ntuples/HHWWgg_DNN/MultiClassifier/HHWWyyDNN_SL_MultiClass_WithMETFix_12LOHH_500epochs_BalanceYields/"
file_string = ".root"

In [3]:
files = ["%s/%s"%(d, f) for f in os.listdir(d) if file_string in f]

In [4]:
# print(files)
# len(files)
for f in files:
    if "Data" in f:
        print("f:",f)

f: /eos/user/a/atishelm/ntuples/HHWWgg_DNN/MultiClassifier/HHWWyyDNN_SL_MultiClass_WithMETFix_12LOHH_500epochs_BalanceYields//Data_2016_HHWWggTag_0_MoreVars_v2.root
f: /eos/user/a/atishelm/ntuples/HHWWgg_DNN/MultiClassifier/HHWWyyDNN_SL_MultiClass_WithMETFix_12LOHH_500epochs_BalanceYields//Data_2017_HHWWggTag_0_MoreVars_v2.root
f: /eos/user/a/atishelm/ntuples/HHWWgg_DNN/MultiClassifier/HHWWyyDNN_SL_MultiClass_WithMETFix_12LOHH_500epochs_BalanceYields//Data_2018_HHWWggTag_0_MoreVars_v2.root


In [5]:
def hist_bin_uncertainty(data, weights, bin_edges):
    """
    The statistical uncertainity per bin of the binned data.
    If there are weights then the uncertainity will be the root of the
    sum of the weights squared.
    If there are no weights (weights = 1) this reduces to the root of
    the number of events.

    Args:
        data: `array`, the data being histogrammed.
        weights: `array`, the associated weights of the `data`.
        bin_edges: `array`, the edges of the bins of the histogram.

    Returns:
        bin_uncertainties: `array`, the statistical uncertainity on the bins.

    Example:
    >>> x = np.array([2,9,4,8])
    >>> w = np.array([0.1,0.2,0.3,0.4])
    >>> edges = [0,5,10]
    >>> hist_bin_uncertainty(x, w, edges)
    array([ 0.31622777,  0.4472136 ])
    >>> hist_bin_uncertainty(x, None, edges)
    array([ 1.41421356,  1.41421356])
    >>> hist_bin_uncertainty(x, np.ones(len(x)), edges)
    array([ 1.41421356,  1.41421356])
    """
    # Bound the data and weights to be within the bin edges
    in_range_index = [idx for idx in range(len(data))
                      if data[idx] > min(bin_edges) and data[idx] < max(bin_edges)]
    in_range_data = np.asarray([data[idx] for idx in in_range_index])

    if weights is None or np.array_equal(weights, np.ones(len(weights))):
        # Default to weights of 1 and thus uncertainty = sqrt(N)
        in_range_weights = np.ones(len(in_range_data))
    else:
        in_range_weights = np.asarray([weights[idx] for idx in in_range_index])

    # Bin the weights with the same binning as the data
    bin_index = np.digitize(in_range_data, bin_edges)
    # N.B.: range(1, bin_edges.size) is used instead of set(bin_index) as if
    # there is a gap in the data such that a bin is skipped no index would appear
    # for it in the set
    binned_weights = np.asarray(
        [in_range_weights[np.where(bin_index == idx)[0]] for idx in range(1, len(bin_edges))])
    bin_uncertainties = np.asarray(
        [np.sqrt(np.sum(np.square(w))) for w in binned_weights])
    return bin_uncertainties

In [13]:
# Continuum_Backgrounds = [
#       'DiPhotonJetsBox_MGG-80toInf_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'GJet_Pt-40toInf_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'TTGG_0Jets_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'TTGJets_TuneCP5_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'TTJets_TuneCP5_extra_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'ttWJets_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'W1JetsToLNu_LHEWpT_150-250_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'W1JetsToLNu_LHEWpT_250-400_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'W1JetsToLNu_LHEWpT_400-inf_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'W1JetsToLNu_LHEWpT_50-150_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'W2JetsToLNu_LHEWpT_150-250_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'W2JetsToLNu_LHEWpT_250-400_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'W2JetsToLNu_LHEWpT_400-inf_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'W2JetsToLNu_LHEWpT_50-150_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'WGGJets_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
#       'WGJJToLNu_EWK_QCD_HHWWggTag_0_MoreVars_kinWeight_noHgg.root',
# ]


useDNNCategories = 0
oneCut = 0


Continuum_Backgrounds = [
    "DiPhotonJetsBox_MGG-80toInf_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "GJet_Pt-40toInf_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "TTGG_0Jets_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "TTGJets_TuneCP5_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "TTJets_TuneCP5_extra_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "W1JetsToLNu_LHEWpT_150-250_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "W1JetsToLNu_LHEWpT_250-400_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "W1JetsToLNu_LHEWpT_400-inf_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "W1JetsToLNu_LHEWpT_50-150_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "W2JetsToLNu_LHEWpT_150-250_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "W2JetsToLNu_LHEWpT_250-400_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "W2JetsToLNu_LHEWpT_400-inf_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "W2JetsToLNu_LHEWpT_50-150_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "WGGJets_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "WGJJToLNu_EWK_QCD_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root",
    "ttWJets_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root"
]


treeDictionary = {
    "DiPhotonJetsBox_MGG-80toInf_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "DiPhotonJetsBox_MGG_80toInf_13TeV_Sherpa_13TeV_HHWWggTag_0_v1",
    "GJet_Pt-40toInf_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "GJet_Pt_40toInf_DoubleEMEnriched_MGG_80toInf_TuneCP5_13TeV_Pythia8_13TeV_HHWWggTag_0_v1",
    "TTGG_0Jets_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "TTGG_0Jets_TuneCP5_13TeV_amcatnlo_madspin_pythia8_13TeV_HHWWggTag_0_v1",
    "TTGJets_TuneCP5_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "TTGJets_TuneCP5_13TeV_amcatnloFXFX_madspin_pythia8_13TeV_HHWWggTag_0_v1",
    "TTJets_TuneCP5_extra_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "TTJets_TuneCP5_13TeV_amcatnloFXFX_pythia8_13TeV_HHWWggTag_0_v1",
    "W1JetsToLNu_LHEWpT_150-250_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "W1JetsToLNu_LHEWpT_150_250_TuneCP5_13TeV_amcnloFXFX_pythia8_13TeV_HHWWggTag_0_v1",
    "W1JetsToLNu_LHEWpT_250-400_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "W1JetsToLNu_LHEWpT_250_400_TuneCP5_13TeV_amcnloFXFX_pythia8_13TeV_HHWWggTag_0_v1",
    "W1JetsToLNu_LHEWpT_400-inf_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "W1JetsToLNu_LHEWpT_400_inf_TuneCP5_13TeV_amcnloFXFX_pythia8_13TeV_HHWWggTag_0_v1",
    "W1JetsToLNu_LHEWpT_50-150_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "W1JetsToLNu_LHEWpT_50_150_TuneCP5_13TeV_amcnloFXFX_pythia8_13TeV_HHWWggTag_0_v1",
    "W2JetsToLNu_LHEWpT_150-250_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "W2JetsToLNu_LHEWpT_150_250_TuneCP5_13TeV_amcnloFXFX_pythia8_13TeV_HHWWggTag_0_v1",
    "W2JetsToLNu_LHEWpT_250-400_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "W2JetsToLNu_LHEWpT_250_400_TuneCP5_13TeV_amcnloFXFX_pythia8_13TeV_HHWWggTag_0_v1",
    "W2JetsToLNu_LHEWpT_400-inf_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "W2JetsToLNu_LHEWpT_400_inf_TuneCP5_13TeV_amcnloFXFX_pythia8_13TeV_HHWWggTag_0_v1",
    "W2JetsToLNu_LHEWpT_50-150_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "W2JetsToLNu_LHEWpT_50_150_TuneCP5_13TeV_amcnloFXFX_pythia8_13TeV_HHWWggTag_0_v1",
    "WGGJets_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "WGGJets_TuneCP5_13TeV_madgraphMLM_pythia8_13TeV_HHWWggTag_0_v1",
    "WGJJToLNu_EWK_QCD_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "WGJJToLNu_EWK_QCD_TuneCP5_13TeV_madgraph_pythia8_13TeV_HHWWggTag_0_v1",
    "ttWJets_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root" : "ttWJets_TuneCP5_13TeV_madgraphMLM_pythia8_13TeV_HHWWggTag_0_v1"
}


Hgg_Backgrounds = [
    
]

    
if(useDNNCategories):
    DNNcuts = [
        [0.1, 0.63],
        [0.63, 0.84],
        [0.84, 0.89],
        [0.89, 1.1]
    ]    
    
else:
    # DNNcuts = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    DNNcuts = [float(i)/20. for i in range(0,20)]
    # DNNcuts = [0.0]

if(oneCut):
    print("Only using one DNN selection")
    DNNcuts = [0.5]
    
# weightTypes = ["weight", "weightKinWeight"]
# weightTypes = ["weightKinWeight_10Sidebands"]
# weightTypes = ["weightKinWeightNoFiducial"]
#weightTypes = ["weightKinWeight_noNegWeights"]
weightTypes = ["weightKinWeight_absValBoth"] # used for DNN 

# xmin, xmax, nbins = 100, 180, 32
xmin, xmax, nbins = 100, 180, 16

for weightType in weightTypes:
    for DNNcut in DNNcuts:
        if(useDNNCategories):
            DNNcut_min, DNNcut_max = DNNcut[0], DNNcut[1]
        else:
            pass
        print("DNNcut:",DNNcut)

        all_vals = np.array([])
        all_weights = np.array([])         
        
        weightedBackgroundPhoMass = np.array([0.] * nbins, dtype = float) ##-- 80 bins 
        for f_i, file in enumerate(files):
            fileEnd = file.split('/')[-1]
            if fileEnd in Continuum_Backgrounds:
                if(f_i%10 == 0):
                    print("on file %s / %s"%(f_i, len(files)))
                #print("on file:",fileEnd)
                f = uproot.open(file)
        #         print("f.keys()",f.keys())
                nTrees = len(f.keys())
                if(nTrees == 1): 
                    #print("Only one tree found:")
                    treeName = f.keys()[0]
                else: 
                    treeName = treeDictionary[fileEnd]
#                 treeName = f.keys()[0] ##-- Assuming one tree
                #treeName = treeDictionary[fileEnd]
                #print("treeName:",treeName)
#                 continue 
                t = f[treeName]
                CMS_hgg_mass = np.array(t["CMS_hgg_mass"].array())
                weight = np.array(t["weight"].array())
                kinWeight = np.array(t["kinWeight"].array())
                evalDNN_HH = np.array(t["evalDNN_HH"].array())
#                 evalDNN_HH = np.array(t["evalDNN"].array())

    #             weightKinWeight = np.multiply(weight, kinWeight)
                if(weightType == "weight"): 
                    weightKinWeight = weight 
                    FIDUCIAL_CUT = [abs(weightKinWeight) < 10.]
                elif(weightType == "weightKinWeight"): 
                    weightKinWeight = np.multiply(weight, kinWeight)
                    FIDUCIAL_CUT = [abs(weightKinWeight) < 10.]
                elif(weightType == "weightKinWeight_10Sidebands"):
                    weightKinWeight = np.multiply(weight, kinWeight)
                    SIDEBANDS = np.logical_or(CMS_hgg_mass < 115, CMS_hgg_mass > 135)
                    FIDUCIAL_CUT = [np.logical_and(abs(weightKinWeight) < 10., SIDEBANDS)]
                elif(weightType == "weightKinWeightNoFiducial"):
                    weightKinWeight = np.multiply(weight, kinWeight)
                    FIDUCIAL_CUT = [np.logical_or(1, 1)]
                elif(weightType == "weightKinWeight_noNegWeights"):
                    weightKinWeight = np.multiply(weight, kinWeight) ##-- Here, it is possible that both are negative. This is the difference between noNegWeights and absValBoth. 
                    FIDUCIAL_CUT = [np.logical_and(abs(weightKinWeight) < 10., weightKinWeight > 0.)]
                elif(weightType == "weightKinWeight_absValBoth"):
                    weightKinWeight = np.multiply(abs(weight), abs(kinWeight))
                    FIDUCIAL_CUT = [abs(weightKinWeight) < 10.]                    

#                 print("FIDUCIAL_CUT:",FIDUCIAL_CUT)
#                 print("len(FIDUCIAL_CUT):",len(FIDUCIAL_CUT))
#                 print("CMS_hgg_mass:",CMS_hgg_mass)
#                 print("len(CMS_hgg_mass):",len(CMS_hgg_mass))
                    
                CMS_hgg_mass = CMS_hgg_mass[FIDUCIAL_CUT]
                weightKinWeight = weightKinWeight[FIDUCIAL_CUT]
                evalDNN_HH = evalDNN_HH[FIDUCIAL_CUT]

                if(useDNNCategories):
                    DNN_CUT_MASK = np.logical_and(evalDNN_HH >= DNNcut_min, evalDNN_HH < DNNcut_max)
                else:
                    DNN_CUT_MASK = [evalDNN_HH > DNNcut]

                CMS_hgg_mass = CMS_hgg_mass[DNN_CUT_MASK]
                weightKinWeight = weightKinWeight[DNN_CUT_MASK]
l
    #             weightedBackgroundPhoMass_thisFile = weightedVals[DNN_CUT_MASK]

                bins = np.linspace(xmin, xmax, nbins + 1)
                binHeights_thisFile, bin_edges = np.histogram(CMS_hgg_mass, weights = weightKinWeight, bins = bins)
            
                all_vals = np.append(all_vals, ak.to_numpy(CMS_hgg_mass))
                all_weights = np.append(all_weights, ak.to_numpy(weightKinWeight))
            
#                 largeWeightsBool = weightKinWeight > 5
#                 if(largeWeightsBool.sum() >= 1):
#                     print("Large weights contained for %s"%(file)) 
#                     # /eos/user/a/atishelm/ntuples/HHWWgg_DNN/MultiClassifier/HHWWyyDNN_SL_MultiClass_WithMETFix_12LOHH_500epochs_BalanceYields//WGJJToLNu_EWK_QCD_HHWWggTag_0_MoreVars_kinWeight_noHgg_v3.root
#                     # weight, kinWeight of *      282 * 0.0020138 * 3119.6814 * making a scary bump. Indicates just due to one MC event. 
            
#                 fig, ax = plt.subplots()
#                 plt.hist(weightKinWeight)
#                 plt.show()
#                 plt.close()
            
    #             print("binHeights_thisFile:",binHeights_thisFile)
    #             print("bin_edges:", bin_edges)
    #             print("binHeights_thisFile:",binHeights_thisFile)
  

                weightedBackgroundPhoMass = np.add(weightedBackgroundPhoMass, binHeights_thisFile)


    #             weightedBackgroundPhoMass = np.append(weightedBackgroundPhoMass, weightedBackgroundPhoMass_thisFile)

        #         plt.hist(weightedVals, bins = bins)
        #         break 


        fig, ax = plt.subplots()
        bins = np.linspace(xmin, xmax, nbins + 1)
        
        
        # sum weights squared for uncertainties 
        binned_MC_stat_uncertainties = hist_bin_uncertainty(all_vals, all_weights, bins)  
        
        # Sqrt per bin for uncertainty
        yerrors = np.sqrt(weightedBackgroundPhoMass)

        bin_centers = [ ((bins[i + 1] - bins[i] ) / 2.) + bins[i] for i in range(len(bins) - 1)] 
        
        ymin = min(weightedBackgroundPhoMass)
        if(ymin > 0): 
            ymin = 0.0
        ymax = max(weightedBackgroundPhoMass) * 1.25 ##-- Make sure there is room for the legend 
        
        plt.hist(bins[:-1], weights = weightedBackgroundPhoMass, bins = bins, label = "Continuum MC: DNN score > %s"%(DNNcut))
        #plt.errorbar(bin_centers, weightedBackgroundPhoMass, yerr = yerrors, linestyle = '', ecolor = 'black')
        plt.errorbar(bin_centers, weightedBackgroundPhoMass, yerr = binned_MC_stat_uncertainties, linestyle = '', ecolor = 'black')
        plt.legend(fontsize = 15)
        plt.ylabel("Weighted Entries [A.U.]", fontsize = 15)
        plt.xlabel(r"$m_{\gamma\gamma}$ [GeV]", fontsize = 15)
        plt.ylim(ymin, ymax)
        
        ##-- Make sure output directory exists 
        outDir = "/eos/user/a/atishelm/www/HIG-21-014/AN_20_165_v7/SM_DNN_MC_Validation_withMETFix/%s/"%(weightType)
    
        if(not os.path.exists(outDir)):
            print("Output directory %s does not exist"%(outDir))
            print("Creating directory")
            os.system("mkdir %s"%(outDir))
            os.system("cp %s/../index.php %s"%(outDir, outDir))
        
        DNNcut_Name = str(DNNcut)
        DNNcut_Name = DNNcut_Name.replace(".", "p")        
        
        plt.savefig("%s/Continuum_MC_DNNgt_%s.png"%(outDir, DNNcut_Name))
        plt.savefig("%s/Continuum_MC_DNNgt_%s.pdf"%(outDir, DNNcut_Name))
#         plt.show()
        plt.close()

print("DONE")

DNNcut: 0.0


/tmp/ipykernel_601/1731418912.py:159: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  CMS_hgg_mass = CMS_hgg_mass[FIDUCIAL_CUT]
/tmp/ipykernel_601/1731418912.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  weightKinWeight = weightKinWeight[FIDUCIAL_CUT]
/tmp/ipykernel_601/1731418912.py:161: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error

on file 30 / 52
on file 40 / 52


/tmp/ipykernel_601/3757062521.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  binned_weights = np.asarray(


DNNcut: 0.05


...

DNNcut: 0.95


/tmp/ipykernel_601/1731418912.py:159: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  CMS_hgg_mass = CMS_hgg_mass[FIDUCIAL_CUT]
/tmp/ipykernel_601/1731418912.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  weightKinWeight = weightKinWeight[FIDUCIAL_CUT]
/tmp/ipykernel_601/1731418912.py:161: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error

on file 30 / 52
on file 40 / 52


/tmp/ipykernel_601/3757062521.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  binned_weights = np.asarray(


DONE


In [20]:
##-- Plot the same but for data sidebands 

useDNNCategories = 0

Data_Files = [
      "Data_2016_HHWWggTag_0_MoreVars_v2.root",
      "Data_2017_HHWWggTag_0_MoreVars_v2.root",
      "Data_2018_HHWWggTag_0_MoreVars_v2.root",
#         "Data_2016_HHWWggTag_0_MoreVars_v2_nodeNumber%s.root"%(nodeNumber),
#         "Data_2017_HHWWggTag_0_MoreVars_v2_nodeNumber%s.root"%(nodeNumber),
#         "Data_2018_HHWWggTag_0_MoreVars_v2_nodeNumber%s.root"%(nodeNumber),    
]

if(useDNNCategories):
    DNNcuts = [
        [0.1, 0.63],
        [0.63, 0.84],
        [0.84, 0.89],
        [0.89, 1.1]
    ]
    
else:
    # DNNcuts = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    DNNcuts = [float(i)/20. for i in range(0,20)]
    # DNNcuts = [0.0]

#DNNcuts = [float(i)/20. for i in range(0,20)]


xmin, xmax, nbins = 100, 180, 16

for DNNcut in DNNcuts:
    if(useDNNCategories):
        DNNcut_min, DNNcut_max = DNNcut[0], DNNcut[1]
    else:
        pass
    print("DNNcut:",DNNcut)    

    Data_Sideband_Diphomass = np.array([0.] * nbins, dtype = float) ##-- 80 bins 
    for file in files:
        fileEnd = file.split('/')[-1]
        if fileEnd in Data_Files:
            
            ##-- Get File 
            print("on file:",fileEnd)
            f = uproot.open(file)
            treeName = f.keys()[0] ##-- Assuming one tree 
            if(nTrees == 1): 
                #print("Only one tree found:")
                treeName = f.keys()[0]
            else: 
                treeName = treeDictionary[fileEnd]            
            t = f[treeName]
            CMS_hgg_mass = np.array(t["CMS_hgg_mass"].array())
            evalDNN_HH = np.array(t["evalDNN_HH"].array())
            
            ##-- Obtain signal region blinded variable distributions
            BLIND_SR_CUT = [np.logical_or(CMS_hgg_mass < 115, CMS_hgg_mass > 135)]
            CMS_hgg_mass = CMS_hgg_mass[BLIND_SR_CUT]
            evalDNN_HH = evalDNN_HH[BLIND_SR_CUT]
            
            ##-- Keep events only which pass DNN score selection 
            #DNN_CUT_MASK = [evalDNN_HH > DNNcut]
            if(useDNNCategories):
                DNN_CUT_MASK = np.logical_and(evalDNN_HH >= DNNcut_min, evalDNN_HH < DNNcut_max)
            else:
                DNN_CUT_MASK = [evalDNN_HH > DNNcut]            
            
            CMS_hgg_mass = CMS_hgg_mass[DNN_CUT_MASK]
            bins = np.linspace(xmin, xmax, nbins + 1)
            binHeights_thisFile, bin_edges = np.histogram(CMS_hgg_mass, bins = bins)

            Data_Sideband_Diphomass = np.add(Data_Sideband_Diphomass, binHeights_thisFile)


    fig, ax = plt.subplots()
    bins = np.linspace(xmin, xmax, nbins + 1)
    yerrors = np.sqrt(Data_Sideband_Diphomass)

    bin_centers = [ ((bins[i + 1] - bins[i] ) / 2.) + bins[i] for i in range(len(bins) - 1)] 

    ymin = min(Data_Sideband_Diphomass)
    if(ymin > 0): 
        ymin = 0.0
    ymax = max(Data_Sideband_Diphomass) * 1.25 ##-- Make sure there is room for the legend     
    
    plt.hist(bins[:-1], weights = Data_Sideband_Diphomass, bins = bins, histtype = 'bar')
#     print("scatter x:",bins[:-1])
#     print("scatter y:",Data_Sideband_Diphomass)
    plt.plot(bin_centers, Data_Sideband_Diphomass, 'o', label = "Data Sideband: DNN score > %s"%(DNNcut), color = 'black')
    plt.errorbar(bin_centers, Data_Sideband_Diphomass, yerr = yerrors, linestyle = '', ecolor = 'black')
    plt.legend(fontsize = 15)
    plt.ylabel("Entries [A.U.]", fontsize = 15)
    plt.xlabel(r"$m_{\gamma\gamma}$ [GeV]", fontsize = 15)
    plt.ylim(ymin, ymax)

    ##-- Make sure output directory exists 
#     outDir = "/eos/user/a/atishelm/www/HHWWgg/AN_v4_MC_Validation/Data_Sideband/"
    #outDir = "/eos/user/a/atishelm/www/HHWWgg/AN_v7_MC_Validation/Data_Sideband/"
    outDir = "/eos/user/a/atishelm/www/HIG-21-014/AN_20_165_v7/SM_DNN_MC_Validation_withMETFix/Data_Sideband/"
    if(not os.path.exists(outDir)):
        print("Output directory %s does not exist"%(outDir))
        print("Creating directory")
        os.system("mkdir %s"%(outDir))
        os.system("cp %s/../index.php %s"%(outDir, outDir))
        
    DNNcut_Name = str(DNNcut)
    DNNcut_Name = DNNcut_Name.replace(".", "p")

    plt.savefig("%s/Data_Sideband_DNNgt_%s.png"%(outDir, DNNcut_Name))
    plt.savefig("%s/Data_Sideband_DNNgt_%s.pdf"%(outDir, DNNcut_Name))
    plt.close()

print("DONE")

DNNcut: 0.0
on file: Data_2016_HHWWggTag_0_MoreVars_v2.root


/tmp/ipykernel_1560/2218762121.py:59: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  CMS_hgg_mass = CMS_hgg_mass[BLIND_SR_CUT]
/tmp/ipykernel_1560/2218762121.py:60: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  evalDNN_HH = evalDNN_HH[BLIND_SR_CUT]
/tmp/ipykernel_1560/2218762121.py:69: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a diff

on file: Data_2017_HHWWggTag_0_MoreVars_v2.root
on file: Data_2018_HHWWggTag_0_MoreVars_v2.root
DNNcut: 0.05
on file: Data_2016_HHWWggTag_0_MoreVars_v2.root


/tmp/ipykernel_1560/2218762121.py:59: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  CMS_hgg_mass = CMS_hgg_mass[BLIND_SR_CUT]
/tmp/ipykernel_1560/2218762121.py:60: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  evalDNN_HH = evalDNN_HH[BLIND_SR_CUT]
/tmp/ipykernel_1560/2218762121.py:69: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a diff

on file: Data_2017_HHWWggTag_0_MoreVars_v2.root
on file: Data_2018_HHWWggTag_0_MoreVars_v2.root
DNNcut: 0.1
on file: Data_2016_HHWWggTag_0_MoreVars_v2.root


...

on file: Data_2017_HHWWggTag_0_MoreVars_v2.root
on file: Data_2018_HHWWggTag_0_MoreVars_v2.root
DNNcut: 0.9
on file: Data_2016_HHWWggTag_0_MoreVars_v2.root


/tmp/ipykernel_1560/2218762121.py:59: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  CMS_hgg_mass = CMS_hgg_mass[BLIND_SR_CUT]
/tmp/ipykernel_1560/2218762121.py:60: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  evalDNN_HH = evalDNN_HH[BLIND_SR_CUT]
/tmp/ipykernel_1560/2218762121.py:69: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a diff

on file: Data_2017_HHWWggTag_0_MoreVars_v2.root
on file: Data_2018_HHWWggTag_0_MoreVars_v2.root
DNNcut: 0.95
on file: Data_2016_HHWWggTag_0_MoreVars_v2.root


/tmp/ipykernel_1560/2218762121.py:59: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  CMS_hgg_mass = CMS_hgg_mass[BLIND_SR_CUT]
/tmp/ipykernel_1560/2218762121.py:60: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  evalDNN_HH = evalDNN_HH[BLIND_SR_CUT]
/tmp/ipykernel_1560/2218762121.py:69: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a diff

on file: Data_2017_HHWWggTag_0_MoreVars_v2.root
on file: Data_2018_HHWWggTag_0_MoreVars_v2.root
DONE
